In [30]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
import csv
from scipy import sparse
import numpy as np
import sklearn.manifold
import sklearn.datasets
from sklearn.decomposition import PCA
import MulticoreTSNE
from sklearn.utils import shuffle

In [162]:
twenty_train = fetch_20newsgroups(shuffle=True, random_state=42)

In [163]:
print (twenty_train.data[1])
print (twenty_train.target_names[1])

From: guykuo@carson.u.washington.edu (Guy Kuo)
Subject: SI Clock Poll - Final Call
Summary: Final call for SI clock reports
Keywords: SI,acceleration,clock,upgrade
Article-I.D.: shelley.1qvfo9INNc3s
Organization: University of Washington
Lines: 11
NNTP-Posting-Host: carson.u.washington.edu

A fair number of brave souls who upgraded their SI clock oscillator have
shared their experiences for this poll. Please send a brief message detailing
your experiences with the procedure. Top speed attained, CPU rated speed,
add on cards and adapters, heat sinks, hour of usage per day, floppy disk
functionality with 800 and 1.4 m floppies are especially requested.

I will be summarizing in the next two days, so please add to the network
knowledge base if you have done the clock upgrade and haven't answered this
poll. Thanks.

Guy Kuo <guykuo@u.washington.edu>

comp.graphics


In [164]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
vocab = count_vect.vocabulary_ 



In [165]:
print (type(vocab))


<class 'dict'>


In [166]:
print (X_train_counts.get_shape())

(11314, 130107)


In [167]:
sparse.save_npz('data_sparse.npz', X_train_counts)


In [168]:
def save_sparse_matrix(m,filename, header=''):
    thefile = open(filename, 'w')
    thefile.write('FIRST_KEY, SECOND_KEY, OCCURENCES \n')
    nonZeros = np.array(m.nonzero()) 
    for entry in range(nonZeros.shape[1]):
        thefile.write("%s,%s,%s\n" % (nonZeros[0, entry], nonZeros[1, entry], m[nonZeros[0, entry], nonZeros[1, entry]]))

In [169]:
save_sparse_matrix(X_train_counts, "matrix.txt")

In [ ]:
with open('dictionary_vocabulary.csv', 'w') as f:  
    w = csv.writer(f)
    f.write('WORD,KEY \n')
    w.writerows(vocab.items())

In [ ]:
digits = sklearn.datasets.load_digits()
orig_X = digits.data
orig_Y = digits.target

every_nth_element = 1 #defult 1 == every element
X = orig_X[0::every_nth_element]
y = orig_Y[0::every_nth_element]

print("Computing t-SNE embedding")
tsne = sklearn.manifold.TSNE(n_components=3, init='pca', random_state=0)
X_tsne = tsne.fit_transform(X)

print(X_tsne)



Computing t-SNE embedding


In [ ]:
print (X_tsne.shape)
print (y_reshaped.shape)
y_reshaped = y.reshape(1797, 1)

merged_X_Y = np.concatenate((X_tsne, y_reshaped), axis=1)

In [ ]:
numpy.savetxt("mnist_small_tsne.csv", merged_X_Y, fmt='%.5e',  delimiter=",", 
              header='X,Y,Z,label', comments='') 

In [38]:
mnist = sklearn.datasets.fetch_mldata('MNIST original')

In [39]:
mnist_X, mnist_Y = shuffle(mnist.data, mnist.target)

In [41]:
full_mnist_data = mnist_X[:10000]
print (full_mnist_data.shape)
full_mnist_labels = mnist_Y[:10000]
print (full_mnist_labels)

(10000, 784)
[ 3.  9.  9. ...,  1.  3.  0.]


In [42]:
pca = PCA(n_components=30)

In [43]:
full_mnist_after_pca = pca.fit_transform(full_mnist_data)

In [44]:
print (full_mnist_after_pca.shape)

(10000, 30)


In [45]:
tsne = MulticoreTSNE.MulticoreTSNE(n_components=3, random_state=0, n_jobs=8)

In [46]:
full_mnist_after_tsne = tsne.fit_transform(full_mnist_after_pca)

In [47]:
full_mnist_labels = full_mnist_labels.reshape(10000, 1)
merged_full_mnist = np.concatenate((full_mnist_after_tsne, full_mnist_labels), axis=1)

In [48]:
print (full_mnist_labels[:100]

[[ 2.]
 [ 1.]
 [ 6.]
 [ 0.]
 [ 8.]
 [ 3.]
 [ 8.]
 [ 9.]
 [ 4.]
 [ 0.]
 [ 5.]
 [ 1.]
 [ 9.]
 [ 9.]
 [ 0.]
 [ 2.]
 [ 9.]
 [ 7.]
 [ 6.]
 [ 6.]
 [ 5.]
 [ 4.]
 [ 7.]
 [ 6.]
 [ 5.]
 [ 0.]
 [ 8.]
 [ 2.]
 [ 5.]
 [ 2.]
 [ 1.]
 [ 5.]
 [ 4.]
 [ 4.]
 [ 3.]
 [ 2.]
 [ 9.]
 [ 8.]
 [ 2.]
 [ 7.]
 [ 3.]
 [ 8.]
 [ 0.]
 [ 9.]
 [ 6.]
 [ 4.]
 [ 1.]
 [ 6.]
 [ 3.]
 [ 9.]
 [ 2.]
 [ 3.]
 [ 9.]
 [ 2.]
 [ 6.]
 [ 2.]
 [ 1.]
 [ 0.]
 [ 9.]
 [ 1.]
 [ 9.]
 [ 9.]
 [ 4.]
 [ 4.]
 [ 3.]
 [ 8.]
 [ 4.]
 [ 3.]
 [ 9.]
 [ 1.]
 [ 9.]
 [ 8.]
 [ 1.]
 [ 4.]
 [ 1.]
 [ 2.]
 [ 9.]
 [ 1.]
 [ 0.]
 [ 3.]
 [ 7.]
 [ 2.]
 [ 8.]
 [ 3.]
 [ 7.]
 [ 7.]
 [ 7.]
 [ 3.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 6.]
 [ 4.]
 [ 7.]
 [ 2.]
 [ 7.]
 [ 3.]
 [ 8.]
 [ 2.]
 [ 9.]
 [ 2.]
 [ 8.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 2.]
 [ 5.]
 [ 4.]
 [ 5.]
 [ 2.]
 [ 3.]
 [ 9.]
 [ 5.]
 [ 2.]
 [ 4.]
 [ 9.]
 [ 8.]
 [ 5.]
 [ 0.]
 [ 4.]
 [ 7.]
 [ 8.]
 [ 8.]
 [ 3.]
 [ 7.]
 [ 3.]
 [ 1.]
 [ 5.]
 [ 1.]
 [ 0.]
 [ 7.]
 [ 3.]
 [ 9.]
 [ 5.]
 [ 9.]
 [ 8.]
 [ 7.]
 [ 4.]
 [ 3.]
 [ 6.]
 [ 3.]
 [ 3.]
 [ 8.]

In [24]:
merged_full_mnist

array([[  7.87529088, -16.35186877, -13.24072328,   0.        ],
       [ 20.84051777, -11.94000424,  -7.51501377,   0.        ],
       [  1.81811332, -29.9204193 ,   3.83347762,   0.        ],
       ..., 
       [  5.4354029 ,  13.25696777,  33.32032126,   1.        ],
       [ 18.49557953,  37.02211976,  11.67647314,   1.        ],
       [ -7.82909439, -25.85147968,  33.51733331,   1.        ]])